In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [100]:
def get_manager_nickname(string):
    manager_info = string.split('"manager_id"')[1]
    manager_info_halves = manager_info.strip("})']").replace("\n","").replace(' ','').replace(':','').replace(',','').strip().split('"nickname"')
    nickname = manager_info_halves[1].replace('"','')
    nickname = nickname.capitalize()
    return nickname
    

In [101]:
def get_manager_id(string):
    manager_info = string.split('"manager_id"')[1]
    manager_info_halves = manager_info.strip("})']").replace("\n","").replace(' ','').replace(':','').replace(',','').strip().split('"nickname"')
    id = manager_info_halves[0]
    return id

In [302]:
class ssf_season:
    """Creates an object representing a single season from a larger dataset.
    Convenient statistics about the season can then be easily accessed as properties.
    
    Parameters:
    year : int
        Year the season began
    standings_data : Pandas dataframe
        Dataframe containing league standings information
    
    """
    masks = {
    '2015': df['season'] == 2015,
    '2016': df['season'] == 2016,
    '2017': df['season'] == 2017,
    '2018': df['season'] == 2018,
    '2019': df['season'] == 2019,
    '2020': df['season'] == 2020,
    '2021': df['season'] == 2021,
    '2022': df['season'] == 2022,
    '2023': df['season'] == 2023
    }
    def __init__(self, year, standings_data, raw_weekly_info = None):
        self.year = year
        self.raw_standings_data = standings_data[masks[str(year)]]
        _df = self.raw_standings_data.copy()
        _df = _df.rename({
                            'name': 'team_name',
                            'team_standings.rank' : 'standings_position',
                            'team_standings.outcome_totals.wins' : 'wins',
                            'team_standings.outcome_totals.losses' : 'losses',
                            'team_standings.outcome_totals.percentage' : 'win_percentage',
                            'team_standings.points_for': 'points_for',
                            'team_standings.points_against' : 'points_against'
                        }, axis = 1)
        # _df['manager_nickname'] = _df['managers.manager'].apply(get_manager_nickname)
        # _df['manager_id'] = _df['managers.manager'].apply(get_manager_id)
        self._season_champion = _df.loc[_df['standings_position']==1, ('team_name','manager_nickname')]
        self._standings_data = _df[['standings_position',
                                    'team_name',
                                    'wins',
                                    'losses',
                                    'win_percentage',
                                    'manager_nickname',
                                    'manager_id']]

        self._pf_pa = _df[['standings_position',
                           'team_name',
                           'manager_nickname',
                           'points_for',
                           'points_against']].copy()
        self._pf_pa['point_differential'] = self._pf_pa['points_for'] - self._pf_pa['points_against']
    @property
    def season_champion(self):
        return [self._season_champion['team_name'][0], self._season_champion['manager_nickname'][0]]

    @property
    def season_champion_manager(self):
        return self._season_champion['manager_nickname'][0]

    @property
    def standings_data(self):
        """
        Simplified standings data for the season object
        """
        return self._standings_data.set_index('standings_position')

    @property
    def standings_data_by_win_percentage(self):
        """
        Standings sorted by win percentage descending
        """
        return self._standings_data.sort_values(by='win_percentage', ascending=False).set_index('standings_position')

    @property
    def pf_pa(self):
        return self._pf_pa.set_index('standings_position')
        

In [270]:
df.columns

Index(['clinched_playoffs', 'division_id', 'draft_grade', 'draft_position',
       'draft_recap_url', 'faab_balance', 'has_draft_grade',
       'league_scoring_type', 'name', 'number_of_moves', 'number_of_trades',
       'team_id', 'team_key', 'url', 'waiver_priority', 'season',
       'managers.manager', 'roster_adds.coverage_type',
       'roster_adds.coverage_value', 'roster_adds.value',
       'team_logos.team_logo', 'team_points.coverage_type',
       'team_points.season', 'team_points.total',
       'team_standings.divisional_outcome_totals.losses',
       'team_standings.divisional_outcome_totals.ties',
       'team_standings.divisional_outcome_totals.wins',
       'team_standings.outcome_totals.losses',
       'team_standings.outcome_totals.percentage',
       'team_standings.outcome_totals.ties',
       'team_standings.outcome_totals.wins', 'team_standings.playoff_seed',
       'team_standings.points_against', 'team_standings.points_for',
       'team_standings.rank', 'team_st

In [ ]:
class ssf_combined_seasons:
    def __init__(self, standings_data, raw_weekly_info = None):
        

In [2]:
df = pd.read_csv('./output/standings_to_date.csv')

In [3]:
df.shape

(126, 38)

In [245]:
df['manager_nickname'] = df['managers.manager'].apply(get_manager_nickname)
df['manager_id'] = df['managers.manager'].apply(get_manager_id)


In [ ]:
df['name'] = df['name'].apply(lambda x: x[1:])

In [252]:
df.loc[df['name'] == "'Thug Bungalow'", 'manager_nickname'] = 'Michael'

In [255]:
df.loc[df['name'] == "'The fishingtons'", 'manager_nickname'] = 'Buddy'

In [262]:
df.loc[df['manager_nickname'] == 'Mattjones', 'manager_nickname'] = 'Jones'

In [263]:
df.loc[df['manager_nickname'] == 'Pd', 'manager_nickname'] = 'Pat'

In [303]:
ssf_15 = ssf_season(2015, df)

In [304]:
ssf_15.season_champion_manager

'Nick'

In [300]:
ssf_16 = ssf_season(2016, df)

In [301]:
ssf_16.pf_pa

,team_name,manager_nickname,points_for,points_against,point_differential
standings_position,,,,,
1,'The fishingtons',Buddy,1318.91,1158.34,160.57
2,'SuperCam That Ho',Lauren,1240.91,1179.19,61.72
3,'The Scientist',Kyle,1317.24,1267.61,49.63
4,"""We're Back""",Nate,1198.14,1208.51,-10.37
5,'who invited canada',Peter,1368.50,1341.00,27.50
6,'The Ice Cold Bruschi',Ricky,1507.68,1257.05,250.63
7,'Thug Bungalow',Michael,1273.57,1262.41,11.16
8,'Victor Cruisin',Matt,1338.66,1306.68,31.98
9,'Immortal Duffmen',Pat,1292.90,1284.79,8.11
